In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digikala-comments-and-products/digikala-comments.csv
/kaggle/input/digikala-comments-and-products/digikala-products.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install hazm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 23.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 80.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not ins

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import hazm
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

In [5]:
data = pd.read_csv('/kaggle/input/digikala-comments-and-products/digikala-comments.csv')

In [6]:
data = data.sample(100000)

In [7]:
data.head()

,id,title,body,created_at,rate,recommendation_status,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate
4967508,45188782,NaN,خوبه,12 بهمن 1401,4.0,recommended,True,662107,NaN,NaN,0,0,دونیک اسپرت,CAYAY,NaN
3290840,54251651,بادی اسپلش مای,بوی خوبی داره\nمن راضی ام,11 مهر 1402,5.0,recommended,True,1819991,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN
1182590,53715764,NaN,خوبه به قیمتش ارزش خرید داره,24 شهریور 1402,4.0,recommended,True,5792153,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN
814626,48413040,متن,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,30 فروردین 1402,4.0,not_recommended,True,7219656,NaN,NaN,0,0,پیتایا,A5ZK9,NaN
628125,40566655,I,اصلا پیشنهاد نمیکنم,23 مهر 1401,1.0,not_recommended,True,1518338,['ندارد'],['کیفیت بد'],0,0,سعادت استایل,EHTVN,NaN


In [8]:
data = data.drop('id'  ,axis=1)

In [9]:
data = data.drop('title'  ,axis=1)

In [10]:
data = data.drop('created_at' ,axis=1)

In [11]:
data = data.drop('recommendation_status' ,axis=1)

In [12]:
data.head()

,body,rate,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate
4967508,خوبه,4.0,True,662107,NaN,NaN,0,0,دونیک اسپرت,CAYAY,NaN
3290840,بوی خوبی داره\nمن راضی ام,5.0,True,1819991,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN
1182590,خوبه به قیمتش ارزش خرید داره,4.0,True,5792153,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN
814626,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,4.0,True,7219656,NaN,NaN,0,0,پیتایا,A5ZK9,NaN
628125,اصلا پیشنهاد نمیکنم,1.0,True,1518338,['ندارد'],['کیفیت بد'],0,0,سعادت استایل,EHTVN,NaN


In [13]:
data = data.drop('is_buyer' ,axis=1)

In [14]:
data = data.drop('product_id' ,axis=1)

In [15]:
data = data.drop('advantages' ,axis=1)

In [16]:
data.head()

,body,rate,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate
4967508,خوبه,4.0,NaN,0,0,دونیک اسپرت,CAYAY,NaN
3290840,بوی خوبی داره\nمن راضی ام,5.0,NaN,0,0,دیجی‌کالا,5A52N,NaN
1182590,خوبه به قیمتش ارزش خرید داره,4.0,NaN,0,0,دیجی‌کالا,5A52N,NaN
814626,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,4.0,NaN,0,0,پیتایا,A5ZK9,NaN
628125,اصلا پیشنهاد نمیکنم,1.0,['کیفیت بد'],0,0,سعادت استایل,EHTVN,NaN


In [17]:
data = data.drop('disadvantages' ,axis=1)

In [18]:
data = data.drop('likes' ,axis=1)

In [19]:
data = data.drop('dislikes' ,axis=1)

In [20]:
data = data.drop('seller_title' ,axis=1)

In [21]:
data = data.drop('seller_code' ,axis=1)

In [22]:
data = data.drop('true_to_size_rate' ,axis=1)

In [23]:
data.head()

,body,rate
4967508,خوبه,4.0
3290840,بوی خوبی داره\nمن راضی ام,5.0
1182590,خوبه به قیمتش ارزش خرید داره,4.0
814626,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,4.0
628125,اصلا پیشنهاد نمیکنم,1.0


In [24]:
def rate_to_label(val):
    if val>= 3.0:
        return 'yes'
    else:
        return 'no'

In [25]:
data['rate'] = data['rate'].apply(rate_to_label)

In [26]:
data = data.rename(columns={"rate": "label_id"})

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 4967508 to 3282959
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   body      99988 non-null   object
 1   label_id  100000 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [28]:
data.head()

,body,label_id
4967508,خوبه,yes
3290840,بوی خوبی داره\nمن راضی ام,yes
1182590,خوبه به قیمتش ارزش خرید داره,yes
814626,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,yes
628125,اصلا پیشنهاد نمیکنم,no


In [29]:
is_nan = data['body'].isna().sum()

In [30]:
print(is_nan)

12


In [31]:
data.dropna(subset=['body'], inplace=True)

In [32]:
print(is_nan)

12


In [33]:
data['comment_len_by_words'] = data['body'].apply(lambda t: len(hazm.word_tokenize(t)))

In [34]:
data

,body,label_id,comment_len_by_words
4967508,خوبه,yes,1
3290840,بوی خوبی داره\nمن راضی ام,yes,6
1182590,خوبه به قیمتش ارزش خرید داره,yes,6
814626,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,yes,10
628125,اصلا پیشنهاد نمیکنم,no,3
...,...,...,...
3472379,بخرید عالیه,yes,2
506880,راضی هستم خیلی خوبه تاثیرش قوی بدون سوزش و سرع...,yes,23
5892700,یه خورده کوچیک بود ولی کیفیت خوب بود,yes,8
4867980,تو تخفیف خوبه,yes,3


In [35]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()

In [36]:
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 1 	Max: 368


In [37]:
def data_gl_than(data, less_than=300.0, greater_than=3.0, col='comment_len_by_words'):
    data_length = data[col].values
    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])
    data_glt_rate = (data_glt / len(data_length)) * 100
    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [38]:
data_gl_than(data, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 73.44% of the whole!


In [39]:
minlim, maxlim = 3, 300

In [40]:
data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [41]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))
fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)
fig.show()

In [42]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path)

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/426k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
data

,body,label_id,comment_len_by_words
0,بوی خوبی داره\nمن راضی ام,yes,6.0
1,خوبه به قیمتش ارزش خرید داره,yes,6.0
2,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,yes,10.0
3,خیلی بد قبلا خریده بودم خوب بود اما دفعه دوم خ...,no,13.0
4,خیلی کیوته و نانازه ولی هنوز استفادش نکردم,yes,8.0
...,...,...,...
73439,مناسب بود و توصیه میکنم .,yes,6.0
73440,به نظرم چیز کارامدی هست.,yes,6.0
73441,راضی هستم خیلی خوبه تاثیرش قوی بدون سوزش و سرع...,yes,23.0
73442,یه خورده کوچیک بود ولی کیفیت خوب بود,yes,8.0


In [44]:
def show_samples(data, n=5):
    for i in range(n):
        print(data[i])
        print()
        
data = data.reset_index(drop=True)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73444 entries, 0 to 73443
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body                  73444 non-null  object 
 1   label_id              73444 non-null  object 
 2   comment_len_by_words  73444 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.7+ MB


In [46]:
data

,body,label_id,comment_len_by_words
0,بوی خوبی داره\nمن راضی ام,yes,6.0
1,خوبه به قیمتش ارزش خرید داره,yes,6.0
2,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,yes,10.0
3,خیلی بد قبلا خریده بودم خوب بود اما دفعه دوم خ...,no,13.0
4,خیلی کیوته و نانازه ولی هنوز استفادش نکردم,yes,8.0
...,...,...,...
73439,مناسب بود و توصیه میکنم .,yes,6.0
73440,به نظرم چیز کارامدی هست.,yes,6.0
73441,راضی هستم خیلی خوبه تاثیرش قوی بدون سوزش و سرع...,yes,23.0
73442,یه خورده کوچیک بود ولی کیفیت خوب بود,yes,8.0


In [47]:
from hazm import stopwords_list

In [48]:
data

,body,label_id,comment_len_by_words
0,بوی خوبی داره\nمن راضی ام,yes,6.0
1,خوبه به قیمتش ارزش خرید داره,yes,6.0
2,خوبه جنسش نازکه و خنکه برا تابستون وبهار بدرد ...,yes,10.0
3,خیلی بد قبلا خریده بودم خوب بود اما دفعه دوم خ...,no,13.0
4,خیلی کیوته و نانازه ولی هنوز استفادش نکردم,yes,8.0
...,...,...,...
73439,مناسب بود و توصیه میکنم .,yes,6.0
73440,به نظرم چیز کارامدی هست.,yes,6.0
73441,راضی هستم خیلی خوبه تاثیرش قوی بدون سوزش و سرع...,yes,23.0
73442,یه خورده کوچیک بود ولی کیفیت خوب بود,yes,8.0


In [49]:
filtered_word_list = data['body'][:]
for word in data['body']:
    if word in stopwords_list():
        filtered_word_list.remove(word)

In [50]:
labels = list(sorted(data['label_id'].unique()))

In [51]:
negative_data = data[data['label_id'] == 'no']
positive_data = data[data['label_id'] == 'yes']

cutting_point = min(len(negative_data), len(positive_data))

if cutting_point <= len(negative_data):
    negative_data = negative_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(positive_data):
    positive_data = positive_data.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([negative_data, positive_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29116 entries, 0 to 29115
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body                  29116 non-null  object 
 1   label_id              29116 non-null  object 
 2   comment_len_by_words  29116 non-null  float64
dtypes: float64(1), object(2)
memory usage: 682.5+ KB


In [52]:
new_data

,body,label_id,comment_len_by_words
0,من برای بار ۵م این کتونی رو سفارش دادم و خیلی ...,yes,26.0
1,ارزش خرید نسبت به قیمت,yes,5.0
2,توی تخفیفات که قیمتش ازمغازه مناسب تر درمیاد ب...,no,20.0
3,من این گازو برای دو قلوهام سفارش دادم واقعا عا...,yes,34.0
4,به نظر من اصلا نخرید ارزش خرید نداره,no,8.0
...,...,...,...
29111,خیلی زود شُل میشه و از بین میره واسه من یه روز...,no,13.0
29112,کتاب بسیار خوبی برای مطالعه هستش,no,6.0
29113,بنظرم توی تخفیف ارزش خرید داره بسته بندی خوبی ...,yes,16.0
29114,عدم تطابق کالا .بعد از ده روز هنوز پول برگردان...,no,12.0


In [53]:
new_data['label_id'] = new_data['label_id'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label_id'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label_id'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['body'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['body'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['body'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(23583, 3)
(2621, 3)
(2912, 3)


In [54]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [56]:
max(data['comment_len_by_words'])

298.0

In [57]:
MAX_LEN = 300
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EPOCHS = 3
EEVERY_EPOCH = 1180
LEARNING_RATE = 2e-5
CLIP = 0.0
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/kaggle/working/pytorch_model.bin'
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [58]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}
print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'no': 0, 'yes': 1}
id2label: {0: 'no', 1: 'yes'}


In [59]:
data['label_id'] = data['label_id'].astype(str)

In [60]:
data.isna().sum()

body                    0
label_id                0
comment_len_by_words    0
dtype: int64

In [61]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })
print(config.to_json_string())

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "no",
    "1": "yes"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "no": 0,
    "yes": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [62]:
idx = np.random.randint(0, len(train))
sample_comment = train.iloc[idx]['body']
sample_label = train.iloc[idx]['label_id']
print(f'Sample: \n{sample_comment}\n{sample_label}')

Sample: 
کیفیت و رنگاش عالی بود، سایزش هم درسته
0


In [63]:
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: کیفیت و رنگاش عالی بود، سایزش هم درسته
   Tokens: کیفیت و رنگاش عالی بود ، سایزش هم درسته
Token IDs: [4589, 1379, 3880, 3472, 4392, 2834, 1348, 12101, 2014, 2820, 21386]


In [64]:
encoding = tokenizer.encode_plus(
    sample_comment,
    max_length=32,
    truncation=True,
    add_special_tokens=True, 
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt', 
)
print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  4589,  1379,  3880,  3472,  4392,  2834,  1348, 12101,  2014,
          2820, 21386,     4,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [65]:
class Dataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Taaghche. """
    def __init__(self, tokenizer, comments, targets=None, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:

            inputs['targets'] = torch.tensor(int(target), dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = Dataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [66]:
label_list = [0, 1]
train_data_loader = create_data_loader(train['body'].to_numpy(), train['label_id'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['body'].to_numpy(), valid['label_id'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['body'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [67]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['comment'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'][0])

dict_keys(['comment', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['از نزدیک خیلی قشنگه ', 'بسیار عالی وبا کیفیت ممنونم.', 'پس از رنگ کردن مو تداوم میبخشد', 'حتما پیشنهاد میکنم. خیلی جذاب و دوست داشتنیه', 'به نظر من این کتاب هم هیجان کافی رو برای نوجوانان به ارمغان میاره هم مطالب آموزنده داره.', 'اصلا خوب نبودن ی تیکه پلاستیکه چسبیه فقط اصلا نخرید اصلا مثل عکساش نبود ', 'واسه این قیمت عالیه', 'جنس نداشت و رنگ\r\nاسیر دست و نام دیجی کالا شدن', 'مدل دانشجو با دقت و زیبا طراحی شده است.', 'خوبه.منتها موها خیلی خشک میشه ک نیاز به ماسک مو هستش', 'بسیار ابزار خوب و باعث شوخی و سرگرمی با کیفیت بالا میباشد. از زحمات مجموعه تقدیر و تشکر میکنم', 'نازک و بدن نما و جنس بد. دوخت بد. رنگ بد. افتزاح.', 'خیلی شور بود نخرید', 'کاملا راضی بودم ممنون از کیفیت و قیمت  عالیتون ', 'به نظرم مسواک خوبیه هم از نظر قیمت هم کیفیت', 'یه بند معمولیه و شیکه جنسش خیلی معمولیه و اگه مراعات کنید براتون خوب کار میکنه']
torch.Size([16, 300])
tensor([    2,  2791,  3620,  3805, 76604,     4,     0,     0,

In [68]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output= self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
#         print(pooled_output[0])
        pooled_output = self.dropout(pooled_output[0])
        logits = self.classifier(pooled_output)
        return logits 

In [69]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Sun Apr 28 20:38:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              26W / 250W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [70]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

pt_model <class '__main__.SentimentModel'>


In [71]:
sample_data_comment = sample_data['comment']
sample_data_input_ids = sample_data['input_ids']
sample_data_attention_mask = sample_data['attention_mask']
sample_data_token_type_ids = sample_data['token_type_ids']
sample_data_targets = sample_data['targets']

sample_data_input_ids = sample_data_input_ids.to(device)
sample_data_attention_mask = sample_data_attention_mask.to(device)
sample_data_token_type_ids = sample_data_token_type_ids.to(device)
sample_data_targets = sample_data_targets.to(device)




outputs = pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids)
_, preds = torch.max(outputs, dim=1)

# print(outputs[:5, :])
# print(preds[:5])

In [72]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs.squeeze(), targets)

            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        _, preds = torch.max(outputs, dim=1)
#         print(outputs)
#         print(targets)
        loss = loss_fn(outputs.squeeze(), targets.to(device))
        
        losses.append(loss.item())

        loss.backward()

#         if clip > 0.0:
#             nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()

        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)
        if step % 1180 == 0:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [73]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/1474 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1180...Train Loss: 0.628627...Train Acc: 0.499...Valid Loss: 0.558197...Valid Acc: 0.500...
Validation loss decreased (inf --> 0.558197).  Saving model ...


Evaluation... :   0%|          | 0/164 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/164 [00:00<?, ?it/s]

Training... :   0%|          | 0/1474 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 3/3...Step: 3540...Train Loss: 0.464328...Train Acc: 0.499...Valid Loss: 0.628071...Valid Acc: 0.500...


Evaluation... :   0%|          | 0/164 [00:00<?, ?it/s]

In [74]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs
     

test_comments = test['body'].to_numpy()
preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/91 [00:00<?, ?it/s]

(2912, 1) (2912, 128, 1)


In [75]:
y_test, y_pred = [label_list.index(label) for label in test['label_id'].values], preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred))

F1: 0.7016966912572359

              precision    recall  f1-score   support

           0       0.73      0.64      0.68      1456
           1       0.68      0.77      0.72      1456

    accuracy                           0.70      2912
   macro avg       0.71      0.70      0.70      2912
weighted avg       0.71      0.70      0.70      2912



In [76]:
concatenated = pd.concat([test, pd.DataFrame(y_pred)], axis="columns")
concatenated = concatenated.rename(columns={0: "predicted"})

In [77]:
concatenated[concatenated['label_id'] != concatenated['predicted']].to_excel('wrongs.xlsx')